In [ ]:
import math, sys, os, numpy as np
from numpy.linalg import norm
from PIL import Image
from matplotlib import pyplot as plt, rcParams, rc
from scipy.ndimage import imread
from skimage.measure import block_reduce
# import cPickle as pickle
from scipy.ndimage.filters import correlate, convolve
from ipywidgets import interact, interactive, fixed
from ipywidgets.widgets import *
rc('animation', html='html5')
rcParams['figure.figsize'] = 3, 6
%precision 4
%matplotlib inline
np.set_printoptions(precision=4, linewidth=100)

In [ ]:
# only run this once to download and save the data

'''from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/')
images, labels = mnist.train.images, mnist.train.labels
images = images.reshape((55000,28,28))
np.savez_compressed('MNIST_data/train', images=images, labels=labels)'''

In [ ]:
def plots(ims, interp=False, titles=None):
    ims = np.array(ims)
    an, ax = ims.min(), ims.max()
    f = plt.figure(figsize=(32,24))
    for i in range(len(ims)):
        sp = f.add_subplot(1, len(ims), i+1)
        if not titles is None: sp.set_title(titles[i], fontsize=18)
        plt.imshow(ims[i], interpolation=None if interp else 'none', vmin=an, vmax=ax)
        
def plot(im, interp=False):
    f = plt.figure(figsize=(3,6), frameon=True)
    plt.imshow(im, interpolation=None if interp else 'none')
    
plt.gray()
plt.close()

In [ ]:
data = np.load('MNIST_data/train.npz')
images = data['images']
labels = data['labels']
n = len(images)
images.shape

In [ ]:
plot(images[0])

In [ ]:
labels[0]

In [ ]:
plots(images[:5], titles=labels[:5])

In [ ]:
top = [[-1,-1,-1],
       [ 1, 1, 1],
       [ 0, 0, 0]]

plot(top)

[Link to kernal examples](http://setosa.io/ev/image-kernels/)

In [ ]:
r = (0, 28)
def zoomin(x1=0, x2=28, y1=0, y2=28):
    plot(images[0, y1:y2, x1:x2])
    
w = interactive(zoomin, x1=r, x2=r, y1=r, y2=r)
w

In [ ]:
k = w.kwargs
dims = np.index_exp[k['y1']:k['y2']:1, k['x1']:k['x2']]
print(images[0][dims])

In [ ]:
corrtop = correlate(images[0], top)
print(corrtop[dims])

In [ ]:
plot(corrtop[dims])

In [ ]:
plot(corrtop)

In [ ]:
np.rot90(top, 1)

In [ ]:
straights = [np.rot90(top, i) for i in range(4)]
plots(straights)

In [ ]:
convtop = convolve(images[0], np.rot90(top, 2))
plot(convtop)
np.allclose(convtop, corrtop)

In [ ]:
br = [[0,   0,   1],
      [0,   1,-1.5],
      [1,-1.5,   0]]

diags = [np.rot90(br, i) for i in range(4)]
plots(diags)

In [ ]:
rots = straights + diags
corrs = [correlate(images[0], rot) for rot in rots]
plots(corrs)

In [ ]:
def pool(im):
    return block_reduce(im, (7,7), np.max)

plots([pool(im) for im in corrs])

In [ ]:
eights = [images[i] for i in range(n) if labels[i] == 8]
ones = [images[i] for i in range(n) if labels[i] == 1]

In [ ]:
plots(eights[:5])
plots(ones[:5])

In [ ]:
pool8 = [np.array([pool(correlate(im, rot)) for im in eights]) for rot in rots]

In [ ]:
plots(pool8[0][:5])

In [ ]:
def normalize(arr):
    return (arr - arr.mean()) / arr.std()

In [ ]:
filts8 = np.array([ims.mean(axis=0) for ims in pool8])
filts8 = normalize(filts8)

In [ ]:
plots(filts8)

In [ ]:
pool1 = [np.array([pool(correlate(im, rot)) for im in ones]) for rot in rots]
filts1 = np.array([ims.mean(axis=0) for ims in pool1])
filts1 = normalize(filts1)

In [ ]:
plots(filts1)

In [ ]:
def pool_corr(im):
    return np.array([pool(correlate(im, rot)) for rot in rots])

In [ ]:
plots(pool_corr(eights[0]))

In [ ]:
def sse(a, b):
    return ((a - b)**2).sum()

def is8(im):
    return 1 if sse(pool_corr(im), filts1) > sse(pool_corr(im), filts8) else 0

In [ ]:
sse(pool_corr(eights[0]), filts1), sse(pool_corr(eights[0]), filts8)

In [ ]:
# Is it an eight?
[np.array([is8(im) for im in ims]).sum() for ims in [eights, ones]]

In [ ]:
# Is it a one, or is it not an eight?
[np.array([1 - is8(im) for im in ims]).sum() for ims in [eights, ones]]

### So how do we make this better?

# Linear Regression Optimization

In [ ]:
import math, sys, os, numpy as np
from numpy.linalg import norm
from numpy.random import random
from PIL import Image
from matplotlib import pyplot as plt, rcParams, animation, rc
from scipy.ndimage import imread
from skimage.measure import block_reduce
# import cPickle as pickle
from scipy.ndimage.filters import correlate, convolve
from tempfile import NamedTemporaryFile
from IPython.display import HTML
from ipywidgets import interact, interactive, fixed
from ipywidgets.widgets import *
rc('animation', html='html5')
rcParams['figure.figsize'] = 3, 3
%precision 4
%matplotlib inline
np.set_printoptions(precision=4, linewidth=100)

In [ ]:
def lin(a, b, x):
    return a*x + b

In [ ]:
a, b = 3, 8

In [ ]:
n = 30
x = random(n)
y = lin(a, b, x)

In [ ]:
x

In [ ]:
y

In [ ]:
plt.scatter(x,y)
plt.xlim(0,)
plt.ylim(0,y.max()+1)
plt.show()

In [ ]:
def sse(y, y_pred):
    return ((y - y_pred)**2).sum()

def loss(y, a, b, x):
    return sse(y, lin(a, b, x))

def avg_loss(y, a, b, x):
    return np.sqrt(loss(y, a, b, x) / n)

In [ ]:
a_guess = -1
b_guess = 1
avg_loss(y, a_guess, b_guess, x)

In [ ]:
lr = 0.01

In [ ]:
def upd():
    global a_guess, b_guess
    y_pred = lin(a_guess, b_guess, x)
    dydb = 2 * (y_pred - y)
    dyda = x * dydb
    a_guess -= lr*dyda.mean()
    b_guess -= lr*dydb.mean()

In [ ]:
fig = plt.figure(dpi=100, figsize=(5,4))
plt.scatter(x,y)
line, = plt.plot(x, lin(a_guess, b_guess, x))
plt.close()

def animate(i):
    line.set_ydata(lin(a_guess, b_guess, x))
    for i in range(10):
        upd()
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(0, 40), interval=100)

In [ ]:
HTML(ani.to_html5_video())